In [6]:
%reset
import os
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import ta

RawStockDataDirectory = 'StockData/Raw/'
ProcessedStockDataDirectory = 'StockData/Processed/'

In [7]:
allData = list()

for stockData in os.listdir(RawStockDataDirectory):
    allData.append(pd.read_csv(RawStockDataDirectory + stockData, usecols=[1,2,3,4,5]))

In [8]:
allData[0]

,Open,High,Low,Close,Volume
0,0.099722,0.100155,0.099722,0.099722,469033600
1,0.094953,0.094953,0.094519,0.094519,175884800
2,0.088015,0.088015,0.087582,0.087582,105728000
3,0.089749,0.090183,0.089749,0.089749,86441600
4,0.092351,0.092785,0.092351,0.092351,73449600
...,...,...,...,...,...
10645,153.789993,156.300003,153.460007,153.830002,87558000
10646,153.699997,154.029999,151.130005,151.600006,56182000
10647,152.809998,153.470001,151.830002,152.869995,47204800
10648,153.559998,154.539993,150.229996,150.589996,53833600


In [9]:
count = 1
for stock in allData:
    # Calculate moving averages
    stock['MA10'] = stock['Close'].rolling(window=10).mean()
    stock['MA20'] = stock['Close'].rolling(window=20).mean()
    stock['MA50'] = stock['Close'].rolling(window=50).mean()
    stock['MA100'] = stock['Close'].rolling(window=100).mean()
    stock['MA200'] = stock['Close'].rolling(window=200).mean()

    # Calculate RSI
    stock['RSI'] = ta.momentum.rsi(stock['Close'], window=14)

    # Calculate Bollinger Bands
    BB = ta.volatility.BollingerBands(stock['Close'])
    stock['BBup'], stock['BBmid'], stock['BBlow'] = BB.bollinger_hband(), BB.bollinger_mavg(), BB.bollinger_lband()

    # Calculate MACD
    MACD = ta.trend.MACD(stock['Close'])
    stock['MACD'], stock['MACDsig'], stock['MACDdiff'] = MACD.macd(), MACD.macd_signal(), MACD.macd_diff()

    # Calculate ROC
    stock['ROC'] = ta.momentum.ROCIndicator(stock['Close']).roc()

    # Calculate KAMA
    stock['KAMA'] = ta.momentum.kama(stock['Close'])

    # Calculate OBV
    stock['OBX'] = ta.volume.OnBalanceVolumeIndicator(close=stock['Close'], volume=stock['Volume']).on_balance_volume()

    # Calculate STOCH
    STOCH = ta.momentum.StochasticOscillator(close=stock['Close'], high=stock['High'], low=stock['Low'])
    stock['STOCH'] = STOCH.stoch()
    stock['STOCHsig'] = STOCH.stoch_signal()

    stock.dropna(inplace=True)
    stock.reset_index(drop=True, inplace=True)

    stock.to_csv(ProcessedStockDataDirectory + "Processed_" + str(count))
    count += 1

In [10]:
allData[0]

,Open,High,Low,Close,Volume,MA10,MA20,MA50,MA100,MA200,...,BBmid,BBlow,MACD,MACDsig,MACDdiff,ROC,KAMA,OBX,STOCH,STOCHsig
0,0.049861,0.050294,0.049861,0.049861,91728000,0.058359,0.064256,0.073569,0.086697,0.091779,...,0.064256,0.050029,-0.006020,-0.004922,-0.001098,-27.673345,0.053552,-87180800,2.173856,0.724619
1,0.052462,0.052896,0.052462,0.052462,94684800,0.057188,0.063388,0.072944,0.086259,0.091542,...,0.063388,0.048521,-0.006071,-0.005152,-0.000919,-22.930674,0.053355,7504000,15.216262,5.796706
2,0.052896,0.053329,0.052896,0.052896,49996800,0.056191,0.062326,0.072337,0.085817,0.091334,...,0.062326,0.047649,-0.006007,-0.005323,-0.000684,-19.736803,0.053286,57500800,18.180428,11.856848
3,0.052896,0.053329,0.052896,0.052896,61129600,0.055367,0.061199,0.071826,0.085396,0.091161,...,0.061199,0.047280,-0.005889,-0.005436,-0.000453,-17.567692,0.053236,118630400,20.512045,17.969578
4,0.057232,0.057665,0.057232,0.057232,45046400,0.054934,0.060483,0.071357,0.085019,0.090998,...,0.060483,0.047314,-0.005383,-0.005425,0.000042,-8.964926,0.053380,163676800,51.431175,30.041216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10446,153.789993,156.300003,153.460007,153.830002,87558000,148.491000,150.640067,141.761182,143.000702,146.823159,...,150.640067,144.779566,1.460911,1.910799,-0.449888,-0.965686,148.666778,224730377800,79.887382,51.139744
10447,153.699997,154.029999,151.130005,151.600006,56182000,148.803001,150.645132,142.088312,143.131319,146.880270,...,150.645132,144.781497,1.524348,1.833508,-0.309161,-1.372715,148.749423,224674195800,61.946961,66.398523
10448,152.809998,153.470001,151.830002,152.869995,47204800,149.199001,150.567921,142.505144,143.281016,146.961002,...,150.567921,144.866416,1.657987,1.798404,-0.140417,0.209762,148.897996,224721400600,72.164083,71.332809
10449,153.559998,154.539993,150.229996,150.589996,53833600,149.318001,150.513000,142.883764,143.361560,147.029140,...,150.513000,144.834612,1.561916,1.751106,-0.189191,1.421067,148.914562,224667567000,53.951593,62.687546
